<a href="https://colab.research.google.com/github/allen44/riiid-test-answer-prediction/blob/main/modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import modules

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import seaborn as sns
from pathlib import Path

%cd /content/drive/MyDrive/Colab Notebooks/riiid-test-answer-prediction/
%pwd

pwd = Path.cwd()

/content/drive/MyDrive/Colab Notebooks/riiid-test-answer-prediction


#Import the data

In [25]:
# #Alternatively, Load a DataFrame of train.csv from pikl for better performance on import than import from csv

# #Define data paths
train_csv_path = Path('/content/drive/MyDrive/Colab Notebooks/riiid-test-answer-prediction/data/train.csv')
train_pkl_path = Path('/content/drive/MyDrive/Colab Notebooks/riiid-test-answer-prediction/data/intermediate/train.pkl.gzip')

Using our insights gained from the EDA, when can import the data from csv with the best parameters for modelling.

In [28]:
# Read data from csv
df = pd.read_csv(train_csv_path, index_col='row_id')

# # Or read data from pikl
# with open( train_pkl_path, 'rb') as f:
#   df = pickle.load(f)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
#After importing as a dataframe, save the dataframe as a binary file, 
# in case the notebook instance crashes, so that we can quickly reload
# the dataframe and resume.
df.to_pickle(train_pkl_path)

In [32]:
df

,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
row_id,,,,,,,,,
0,0,115,5692,0,1,3,1,NaN,<NA>
1,56943,115,5716,0,2,2,1,37000.0,False
2,118363,115,128,0,0,0,1,55000.0,False
3,131167,115,7860,0,3,0,1,19000.0,False
4,137965,115,7922,0,4,1,1,11000.0,False
...,...,...,...,...,...,...,...,...,...
101230327,428564420,2147482888,3586,0,22,0,1,18000.0,True
101230328,428585000,2147482888,6341,0,23,3,1,14000.0,True
101230329,428613475,2147482888,4212,0,24,3,1,14000.0,True


In [33]:
df.dtypes

timestamp                            int64
user_id                           category
content_id                        category
content_type_id                   category
task_container_id                 category
user_answer                       category
answered_correctly                category
prior_question_elapsed_time        float32
prior_question_had_explanation     boolean
dtype: object

# Make Train Test Splits

In [36]:
# define target variable, y, and independant variables, X

X = df.drop(columns=['answered_correctly'])
y = df['answered_correctly']
del df

X.shape, y.shape


((101230332, 8), (101230332,))

In [35]:
# Make Train Test Splits

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((80984265, 8), (20246067, 8), (80984265,), (20246067,))

# Preprocessing: Transform data to fit classifier

Some classifier require variables in a certian form. Some classifiers cannot handle NA values. We will fix those in this section.


In [37]:
# Count NA values
X_train.isna().sum()

timestamp                               0
user_id                                 0
content_id                              0
content_type_id                         0
task_container_id                       0
user_answer                             0
prior_question_elapsed_time       1881194
prior_question_had_explanation     313799
dtype: int64

Even though most data is not NA, recall that the data description showed that some data is "null" but were encoded as '-1' or other values. We'll encode them as NA instead.

In [45]:
# Convert Null values to NA

encoded_X_train = X_train.copy()
print('Before:\n', encoded_X_train.isna().sum())

# content_type_id: "1 if the event was the user watching a lecture."
encoded_X_train['content_type_id'] = encoded_X_train['content_type_id'].mask(encoded_X_train['content_type_id']=='1')


# user_answer: " Read -1 as null, for lectures."
encoded_X_train['user_answer'] = encoded_X_train['user_answer'].mask(encoded_X_train['user_answer']=='-1')

# # answered_correctly Read -1 as null, for lectures
# encoded_X_train['answered_correctly'] = encoded_X_train['answered_correctly'].mask(encoded_X_train['answered_correctly']==-1)


# Count the NA entries
print('\nAfter:\n', encoded_X_train.isna().sum())

Before:
 timestamp                               0
user_id                                 0
content_id                              0
content_type_id                         0
task_container_id                       0
user_answer                             0
prior_question_elapsed_time       1881194
prior_question_had_explanation     313799
dtype: int64

After:
 timestamp                               0
user_id                                 0
content_id                              0
content_type_id                   1567395
task_container_id                       0
user_answer                       1567395
prior_question_elapsed_time       1881194
prior_question_had_explanation     313799
dtype: int64


In the before and after printouts, we see the effect of masking the null-encoded values, and replacing them with NA.

## Drop NA rows 

Based on our earlier EDA, we have enough information to make informed decision on how to handle NA values.



In [46]:
encoded_X_train.shape

(80984265, 8)

In [47]:
encoded_X_train_no_na = encoded_X_train.dropna().reset_index(drop=True)
encoded_X_train_no_na.isna().sum()

timestamp                         0
user_id                           0
content_id                        0
content_type_id                   0
task_container_id                 0
user_answer                       0
prior_question_elapsed_time       0
prior_question_had_explanation    0
dtype: int64

In [48]:
encoded_X_train_no_na.shape

(79103071, 8)

We see that the .shape attribute shows that about 2% of the entries were dropped.

## Drop features that don't predict the target variable

In the EDA, we learned that user_id doesn't predict the target variable.

In [49]:
# Drop user_id
encoded_X_train_no_na = encoded_X_train_no_na.drop(columns='user_id').reset_index(drop=True)
encoded_X_train_no_na

,timestamp,content_id,content_type_id,task_container_id,user_answer,prior_question_elapsed_time,prior_question_had_explanation
0,14966094946,525,0,526,0,12000.0,True
1,2382277090,4978,0,213,1,22000.0,True
2,15766058493,2332,0,238,0,21000.0,True
3,959500778,2912,0,108,3,36000.0,True
4,1321791120,370,0,140,0,15000.0,True
...,...,...,...,...,...,...,...
79103066,15045271680,5255,0,413,1,29000.0,True
79103067,1804368270,7859,0,102,1,21000.0,True
79103068,798775296,6668,0,17,0,28000.0,True
79103069,2094072,7928,0,77,3,18000.0,True


## Encode Dummy Variables

The categorical variables need to be one-hot-encoded for most of the models to work correctly.

The numerical variables may need to be standardized for some models.


In [19]:
# X_t = pd.get_dummies(encoded_X_train_no_na, drop_first=False, dtype=bool)
X_t = pd.get_dummies(encoded_X_train_no_na, drop_first=True, dtype=bool)
X_t

,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,prior_question_elapsed_time,prior_question_had_explanation_True
row_id,,,,,,,,
0,0,115,5692,0,1,3,NaN,False
1000,15092942969,13134,9155,0,299,3,27666.0,True
2000,1623047964,24418,476,0,112,0,15000.0,True
3000,4746311751,24418,5168,0,1000,1,6000.0,True
4000,5617496174,24418,3746,0,1747,3,27000.0,True
...,...,...,...,...,...,...,...,...
101226000,3254843665,2147413636,3966,0,1278,2,12000.0,True
101227000,4930309457,2147413636,3076,0,1900,3,26000.0,True
101228000,14261958759,2147419988,3177,0,248,1,18000.0,True


We see the row count is unchanged, but now there's many more columns due to how we changed all the categorical data into numerical data.

In [20]:
X_t.shape

(101231, 8)

###Alternatively, encode with sklearn's OneHotEncoder


In [21]:
from sklearn.preprocessing import OneHotEncoder

X_train_cat = encoded_X_train_no_na.select_dtypes(['category', 'bool'])
X_train_num = encoded_X_train_no_na.select_dtypes(['int64', 'float64'])

# Create the encoder.
encoder = OneHotEncoder(handle_unknown="ignore")

# Fit on X_train_cat and then Transform X_train_cat
X_train_cat = encoder.fit_transform(X_train_cat)
X_train_cat.shape

(98814, 0)

In [22]:
X_train_cat

<98814x0 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>

### Standardize the numerical variables.



In [24]:
X.dtypes

timestamp                           int64
user_id                             int64
content_id                          int64
content_type_id                     int64
task_container_id                   int64
user_answer                         int64
prior_question_elapsed_time       float64
prior_question_had_explanation     object
dtype: object

In [23]:
# Check for normality on numerical features
from scipy.stats import kstest

X_num = X.select_dtypes(exclude=['category', 'bool'])
numerical_columns = X_num.columns

for feature in X_num.columns:
  statistic, p = kstest(X_num[feature], 'norm')
  print(f"\n{feature}:\nStatistic: {statistic:0.5f}, p_value: {p:0.5f}")
  print('Probably gaussian.') if p>0.05 else print('Probably not gaussian.') 


timestamp:
Statistic: 0.99598, p_value: 0.00000
Probably not gaussian.

user_id:
Statistic: 1.00000, p_value: 0.00000
Probably not gaussian.

content_id:
Statistic: 0.99927, p_value: 0.00000
Probably not gaussian.

content_type_id:
Statistic: 0.50000, p_value: 0.00000
Probably not gaussian.

task_container_id:
Statistic: 0.98727, p_value: 0.00000
Probably not gaussian.

user_answer:
Statistic: 0.54300, p_value: 0.00000
Probably not gaussian.

prior_question_elapsed_time:
Statistic: nan, p_value: nan
Probably not gaussian.


/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1827: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= np.asarray(_b)) & cond0


TypeError: ignored

We see that none of the numerical features are normally distributed. Let's standardize them using the StandardScaler.

In [ ]:
from sklearn.preprocessing import StandardScaler
 
# Instantiate scaler object
scaler = StandardScaler()

# Fit: Compute the mean and std to be used for later scaling.
# And tranform
X_new = scaler.fit_transform(X_num)
X_new

In [ ]:
# Check the old vs new mean
np.mean(X_num, axis=0), np.mean(X_new, axis=0)

In [ ]:
# Check the old vs new standard deviation
np.std(X_num, axis=0), np.std(X_new, axis=0)

As expected, the X_new means are zero, and the standard deviations are 1.

### Combine the transformed features into one transformed dataframe.

In [ ]:
#Make sure the shapes are compatible
X_t.shape, X_new.shape


In [ ]:
# Check index on X_t
X_t.index

In [ ]:
# Make a dataframe for X_new
X_new_df = pd.DataFrame(X_new, columns=numerical_columns)
X_new_df

In [ ]:
X_new_df.isna().sum()

In [ ]:
X_new_df.dtypes

In [ ]:
X_t[numerical_columns].dtypes

In [ ]:
# Select all numerical columns from X_new

numerical_columns = X_new_df.select_dtypes(include=['int64', 'float64']).columns
numerical_columns

In [ ]:
# Add transformed numerical columns
for num_feature in numerical_columns:
  X_t[num_feature] = X_new_df[num_feature]
X_t[numerical_columns]

In [ ]:
X_t.dtypes

In [ ]:
X_t.shape

# Baseline Modelling

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import plot_confusion_matrix

# Instantiate classifier object
log_reg = LogisticRegression()

# Fit classifier
log_reg.fit(X_train, y_train)

# Make predictions on the train and test sets
y_train_preds = log_reg.predict_proba(X_train)[:,1]
y_test_preds = log_reg.predict_proba(X_test)[:,1]

#Score the predictions
log_reg_train_roc_score = roc_auc_score(y_train, y_train_preds)
log_reg_test_roc_score = roc_auc_score(y_test, y_test_preds)

print(f"Training roc_auc_score: {log_reg_train_roc_score:0.5f}")
print(f"Testing roc_auc_score: {log_reg_test_roc_score:0.5f}")

# Plot normalized confusion matrix
disp = plot_confusion_matrix(log_reg, X_test, y_test,
                              cmap=plt.cm.Blues,
                             normalize='true')
disp.ax_.set_title("Normalized confusion matrix")
plt.show()

That's a pretty bad score for a baseline model. It's good in training, and terrible in test.


## LightGBM

In [ ]:
from lightgbm import LGBMClassifier

# Instantiate the classifier object
lgbm = LGBMClassifier(
    num_leaves=31, 
    max_depth= 2, 
    n_estimators = 25, 
    min_child_samples = 1000, 
    subsample=0.7, 
    subsample_freq=5,
    n_jobs= -1,
    is_higher_better = True,
    first_metric_only = True
)

# Fit the classifier object to the train data
lgbm.fit(X_train, y_train)

# Make predictions on the train and test sets
y_train_preds = lgbm.predict_proba(X_train)[:,1]
y_test_preds = lgbm.predict_proba(X_test)[:,1]

#Score the predictions
lgbm_train_roc_score = roc_auc_score(y_train, y_train_preds)
lgbm_test_roc_score = roc_auc_score(y_test, y_test_preds)

print(f"Training roc_auc_score: {lgbm_train_roc_score:0.5f}")
print(f"Testing roc_auc_score: {lgbm_test_roc_score:0.5f}")

# Plot normalized confusion matrix
disp = plot_confusion_matrix(lgbm, X_test, y_test,
                              cmap=plt.cm.Blues,
                             normalize='true')
disp.ax_.set_title("Normalized confusion matrix")
plt.show()

We see that the LGBM baseline is better than the logistic regression model baseline.

## K-Nearest-Neighbors Classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Instantiate the classifier object
knn = KNeighborsClassifier(n_neighbors=3)

# Fit the classifier object to the train data
knn.fit(X_train, y_train)

# Make predictions on the train and test sets
y_train_preds = knn.predict_proba(X_train)[:,1]
y_test_preds = knn.predict_proba(X_test)[:,1]

#Score the predictions
knn_train_roc_score = roc_auc_score(y_train, y_train_preds)
knn_test_roc_score = roc_auc_score(y_test, y_test_preds)

print(f"Training roc_auc_score: {knn_train_roc_score:0.5f}")
print(f"Testing roc_auc_score: {knn_test_roc_score:0.5f}")

# Plot normalized confusion matrix
disp = plot_confusion_matrix(knn, X_test, y_test,
                              cmap=plt.cm.Blues,
                             normalize='true')
disp.ax_.set_title("Normalized confusion matrix")
plt.show()

## Section Summary

We tested three baseline models: Logistic Regression, LightGBM, and K-Nearest Neighbors. Each model is from a different class of models and they each have their trade-offs and assumptions.

Logistic Regression performed well intraining and poorly in testing.

K-Nearest-Neighbors performed great in training but worst of all three models on the test data. This is evidence that the model baseline tuning is over-fitting on noise in the training data.

LightGBM performed the worst with the baseline hyperparameters. It assigned the target variable the same value for all inputs. It did not learn anything from training. I'll have to adjust the code on this model.



We see that the baseline K-Nearest-Neighbors Classifier did quite well on the training set, but very poorly on the test set. KNN takes the longest to run. I'll have to adjust the hyper parameters or do some feature engineering to reduce the number of columns. Also, as the KNN took about 20 times longer to run than the other models, it makes sense that it's mch better.


In [ ]:
#Training and test roc_scores
print(f"log_reg_train_roc_score: {log_reg_train_roc_score:0.5f}")
print(f"log_reg_test_roc_score: {log_reg_test_roc_score:0.5f}")

print(f"lgbm_train_roc_score: {lgbm_train_roc_score:0.5f}")
print(f"lgbm_test_roc_score: {lgbm_test_roc_score:0.5f}")

print(f"knn_train_roc_score: {knn_train_roc_score:0.5f}")
print(f"knn_test_roc_score: {knn_test_roc_score:0.5f}")

To do:
after chi square, add a visual

If the cat var is not robust, it might not contribute much. Make a chart to see this.

Use a logistic regression model:

fOR each one var, convert to dummy var, and build a model to pred the target,
Get_dummy()
use train_test_split() (20% test, 80% train), dont train model on test data split. (high bias, high variance tradeoff: find the optimum, buy evaluating the model on the test split.)
Choose a model:

Based on advantages and disadvantages of each model, and it's assumptions match the data set.
Start with one model from each family
Decision tree - no further treatment of model needed. Maximizes information. Don't need to normalize/scale. Watch out for over fitting with a complex decision tree.
KNN model -
Boosting model like random forest Will have a p val for each category. ANOVA on each cat in logistic regression.
Include a brief writeup of the advantages, disadvantages, and assumptions on each model.

https://scikit-learn.org/stable/modules/tree.html#classification

Measure performance of the model with confusion matrix.

Next meeting at Jan 8, 2pm HST (4pm PST)